# Face Recognition Assignment #1

## Problem Statement 

We intend to perform face recognition. Face recognition means that for a given image
you can tell the subject id. Our database of subject is very simple. It has 40 subjects.
Below we will show the needed steps to achieve the goal of the assignment.

### Step 1: Download the dataset and understand the format 

In [126]:
from os import listdir
from PIL import Image as PImage

def loadImages(path):
    # return array of images
    foldersList = listdir(path)
    loadedImages = []
    for folder in foldersList :
        imagesList = listdir(path+folder)
        for image in imagesList:
            img = PImage.open(path +folder+'/'+ image)
            loadedImages.append(img)
    return loadedImages

In [132]:
path = "./orl_faces/"
# your images in an array
imgs = loadImages(path)

### Step 2: Generate the Data Matrix and the Label vector 

In [148]:
import numpy as np
dataMatrix = np.arange(4121600).reshape(400,10304)
j=0
label = []
for i in range(0,400) :
    if(i%10 == 0):
        j = j+1
    dataMatrix[i] = np.array(imgs[i]).flatten()
    label.append(j)
label

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 22,
 22,
 22,
 22,
 22,
 22,
 22,
 22,


### Step 3: Splitting the dataset into training and test sets

In [189]:
trainSet=np.arange(200*10304).reshape(200,10304)
testSet=np.arange(200*10304).reshape(200,10304)
trainLabel=[]
testLabel=[]
j,k=0,0
for i in range(0,400):
    if(i%2==0):
        testSet[j]=dataMatrix[i]
        testLabel.append(label[i])
        j+=1
    else:
        trainSet[k]=dataMatrix[i]
        trainLabel.append(label[i])
        k+=1


### Step 4: Classification using PCA

In [232]:
dataMean=np.mean(trainSet,axis=0)
centerDataMatrix=trainSet-dataMean
covMatrix=(1/200)*(centerDataMatrix @ centerDataMatrix.T)
eigVal,eigVectMatrix=np.linalg.eigh(covMatrix)
eigVal=np.flip(eigVal,axis=0)
eigVectMatrix=np.flip(eigVectMatrix,axis=0)


In [233]:
def computeDimensionality(eigVal,threshold):
    totalDataVariance=np.sum(eigVal)
    varianceFraction,numDim,eigValSum=0,1,0
    
    while(varianceFraction<threshold):
        eigValSum+=eigVal[numDim-1]
        varianceFraction=eigValSum/totalDataVariance
        numDim+=1
    return numDim

In [234]:
dimAlpha1=computeDimensionality(eigVal,0.8)
dimAlpha2=computeDimensionality(eigVal,0.85)
dimAlpha3=computeDimensionality(eigVal,0.9)
dimAlpha4=computeDimensionality(eigVal,0.95)


In [237]:
projMat1=eigVectMatrix[:,0:dimAlpha1]
projMat2=eigVectMatrix[:,0:dimAlpha2]
projMat3=eigVectMatrix[:,0:dimAlpha3]
projMat4=eigVectMatrix[:,0:dimAlpha4]


In [241]:
reducedDimTrain1=projMat1.T @ trainSet
reducedDimTrain2=projMat2.T @ trainSet
reducedDimTrain3=projMat3.T @ trainSet
reducedDimTrain4=projMat4.T @ trainSet


In [250]:
reducedDimTest1=projMat1.T @ testSet
reducedDimTest2=projMat2.T @ testSet
reducedDimTest3=projMat3.T @ testSet
reducedDimTest4=projMat4.T @ testSet


### Step 5: Classification using LDA

In [263]:
dataClassList=[]
for i in range(0,401):
    if(i%10==0 and i!=0):
        dataClassList.append(dataMatrix[i-10:i,:])

10

In [279]:
mean=np.mean(dataClassList,axis=1)

(40, 10304)

In [281]:
S=

10304